In [7]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, BatchSampler


In [9]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
%matplotlib inline
BATCH_SIZE = 19
import open_clip
from sklearn import metrics

In [11]:
import pandas as pd
df = pd.read_csv('./label.csv', index_col=0)

In [13]:
df

,image,label,text,label_index
0,./train/bad joint/8.png,bad joint,a photo of door frame,0
1,./train/bad joint/9.png,bad joint,a photo of door frame,0
2,./train/bad joint/14.png,bad joint,a photo of door frame,0
3,./train/bad joint/15.png,bad joint,a photo of door frame,0
4,./train/bad joint/17.png,bad joint,a photo of door frame,0
...,...,...,...,...
3451,./train/damage/379.png,damage,a hole or crack in the wallpaper,18
3452,./train/damage/423.png,damage,a hole or crack in the wallpaper,18
3453,./train/damage/345.png,damage,a hole or crack in the wallpaper,18
3454,./train/damage/351.png,damage,a hole or crack in the wallpaper,18


In [14]:
df.label.unique()

array(['bad joint', 'mopholder', 'crack', 'Furniture', 'molding',
       'window frame', 'gap', 'black spot', 'wailing', 'plaster', 'mold',
       'half spot', 'cotton defect', 'wrinkle', 'twist', 'rust pollution',
       'piece', 'contamination', 'damage'], dtype=object)

In [15]:
df.text.unique()

array(['a photo of door frame', 'the baseboard with a crack',
       'wall paper is coming off', 'crack or wrinkle at the wall',
       'ceiling is coming off', 'crack near electric outlet',
       'wall paper is coming off seriously', 'a hole at the ceiling',
       'wall paper is cut', 'crack on the plaster board',
       'stains on the wall', 'red dots on the wall',
       'a protruding part of the wallpaper',
       'wrinkle on the ceiling or wall', 'The defective seam wrinkles',
       'wall with a brown spot', 'nail on the wall',
       'contamination on the wall', 'a hole or crack in the wallpaper'],
      dtype=object)

In [16]:
text_dict = {'a photo of door frame' : 'the door frame with a crack', 
 'the baseboard with a crack' : 'the baseboard with a crack', 
 'wall paper is coming off' : 'wall paper is coming off at the corner',
 'crack or wrinkle at the wall' : 'the crack with drawer and wall',
 'ceiling is coming off' : 'corner of the ceiling',
 'crack near electric outlet' : 'crack near electric outlet',
 'wall paper is coming off seriously' : 'serious crack at the corner',
 'a hole at the ceiling' : 'a hole at the ceiling',
 'wall paper is cut' : 'wall paper is cut',
 'crack on the plaster board' : 'crack on the ceiling',
 'stains on the wall' : 'stains on the wall',
 'red dots on the wall' : 'red dots or blue dots on the wall',
 'a protruding part of the wallpaper' : 'a protruding part of the wallpaper',
 'wrinkle on the ceiling or wall' : 'wrinkle on the ceiling or wall',
 'The defective seam wrinkles' : 'seam wrinkles around the corner',
 'wall with a brown spot' : 'wall with a brown spot',
 'nail on the wall' : 'nail on the wall',
 'contamination on the wall' : 'contamination on the wall',
 'a hole or crack in the wallpaper' : 'a hole or crack in the wallpaper'
}

In [17]:
df['text'] = df['text'].map(text_dict)

In [10]:
# def rename_text(string):
#     return 'a photo of ' + string

In [11]:
# df['text'] = df['label'].apply(rename_text)

In [12]:
# def rename_text(string):
#     if string[:7] == 'a photo':
#         return string
#     else:
#         return 'a photo of ' + string


In [13]:
# df['text'] = df['text'].apply(rename_text)

In [18]:
text_label = list(df.text.unique())

In [19]:
text_label

['the door frame with a crack',
 'the baseboard with a crack',
 'wall paper is coming off at the corner',
 'the crack with drawer and wall',
 'corner of the ceiling',
 'crack near electric outlet',
 'serious crack at the corner',
 'a hole at the ceiling',
 'wall paper is cut',
 'crack on the ceiling',
 'stains on the wall',
 'red dots or blue dots on the wall',
 'a protruding part of the wallpaper',
 'wrinkle on the ceiling or wall',
 'seam wrinkles around the corner',
 'wall with a brown spot',
 'nail on the wall',
 'contamination on the wall',
 'a hole or crack in the wallpaper']

In [20]:
len(text_label)

19

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df, df['label_index'], test_size=0.33, random_state=777, stratify=df['label_index'])

In [23]:
x_train

,image,label,text,label_index
1222,./train/twist/165.png,twist,seam wrinkles around the corner,14
331,./train/mopholder/91.png,mopholder,the baseboard with a crack,1
3416,./train/damage/225.png,damage,a hole or crack in the wallpaper,18
1569,./train/contamination/563.png,contamination,contamination on the wall,17
3299,./train/damage/1214.png,damage,a hole or crack in the wallpaper,18
...,...,...,...,...
2396,./train/damage/402.png,damage,a hole or crack in the wallpaper,18
571,./train/window frame/128.png,window frame,crack near electric outlet,5
1539,./train/contamination/549.png,contamination,contamination on the wall,17
2910,./train/damage/1129.png,damage,a hole or crack in the wallpaper,18


## dataset 정의와 batchsampler 정의

In [16]:
from torch.utils.data import Dataset, DataLoader
class FlawDataset(Dataset):
    def __init__(self, csv_df, transform):
        self.csv_df = csv_df
        self.img_list = []
        self.transform =transform
        for img_path in self.csv_df['image']:
            self.img_list.append(Image.open(img_path))
        
        
        self.text = self.csv_df['text'].to_list()
        self.label = self.csv_df['label_index'].to_list()
        
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img = self.img_list[idx]
        text = self.text[idx]
        label = self.label[idx]
        
        img = self.transform(img)
        
        return img, text, label

In [17]:
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

In [18]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [19]:
device='cuda'

In [20]:
model, preprocess = clip.load('ViT-L/14@336px', device=device, jit=False)

In [21]:
preprocess

Compose(
    Resize(size=336, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(336, 336))
    <function _convert_image_to_rgb at 0x7f38ad527370>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [22]:
def _convert_image_to_rgb(image):
    return image.convert("RGB")

In [23]:
from torchvision import transforms
from torchvision.transforms import InterpolationMode
train_preprocess = transforms.Compose([
    transforms.RandomResizedCrop(size=(336, 336), interpolation=InterpolationMode.BICUBIC, scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    transforms.autoaugment.TrivialAugmentWide(interpolation=InterpolationMode.BILINEAR),
    #_convert_image_to_rgb,
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)),
    transforms.RandomErasing(p=0.1)
]
)

val_preprocess = transforms.Compose([
    transforms.Resize(size=(336, 336), interpolation=InterpolationMode.BICUBIC, max_size=None, antialias=None),
    transforms.CenterCrop(size=(336, 336)),
    #_convert_image_to_rgb,
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711)),
]
)

In [28]:
process = transforms.Compose([
    transforms.Resize(size=336, interpolation=InterpolationMode.BICUBIC, max_size=None, antialias=None),
    transforms.ToTensor()
])

In [24]:
train_dataset = FlawDataset(x_train, train_preprocess)
test_dataset = FlawDataset(x_test, val_preprocess)

In [25]:
BATCH_SIZE = 19

In [26]:
train_labels = torch.tensor([item[2] for item in train_dataset])
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)
#train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle = True, drop_last = True)

test_labels = torch.tensor([item[2] for item in test_dataset])
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)
#test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle = False, drop_last = True)

In [27]:
EPOCH = 40

In [28]:
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=1e-6,betas=(0.9,0.98),eps=1e-6,weight_decay=0.05)
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
#optimizer = optim.SGD(model.parameters(), lr=1e-5, momentum=0.9, weight_decay = 1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [29]:
best_f1 = 1e-5
best_ep = -1
ans = x_test['label_index'].to_list()
best_te_loss = 1e5
f1_ls = []


for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
        #print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step
    
    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step
        
    preds = []
    for image in x_test['image']:
        image = preprocess(Image.open(image)).unsqueeze(0).to(device)
        text = clip.tokenize(text_label).to(device)

        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)

            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu()
            prob_idx = probs.argmax()
            preds.append(prob_idx)
            
            
    ans = x_test['label_index'].to_list()
    print(metrics.accuracy_score(ans, preds))
    f1_acc = metrics.f1_score(ans, preds, average = 'macro')
    print(f1_acc)
    f1_ls.append(f1_acc)
            
            

    if best_f1 < f1_acc:
        best_f1_acc = f1_acc
        best_ep = epoch
        torch.save(model.state_dict(), "./best_model_change_val336gpuno2f.pt")
        
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "./lasttr224_model25.pt")

running epoch 0, best test loss 100000.0 after epoch -1


  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

0.7239263803680982
0.5706872079875117
epoch 0, tr_loss 0.9229514382102273, te_loss 1.0137736002604167
running epoch 1, best test loss 100000.0 after epoch 0


  0%|          | 0/121 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

0.8860648553900088
0.792198876281802
epoch 38, tr_loss 0.055849400433627044, te_loss 1.1948038736979167
running epoch 39, best test loss 100000.0 after epoch 38


  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

0.8869412795793163
0.7928703908931733
epoch 39, tr_loss 0.05190930878820498, te_loss 1.2280354817708334


In [ ]:
!ls

In [30]:
model.load_state_dict(torch.load("./best_model_change_val336gpuno2f.pt"))

<All keys matched successfully>

## validate model

In [31]:
import glob
test_ls = glob.glob('./test/*.png')

In [32]:
test_ls.sort()

In [33]:
from sklearn import metrics

In [34]:
test_ls

['./test/000.png',
 './test/001.png',
 './test/002.png',
 './test/003.png',
 './test/004.png',
 './test/005.png',
 './test/006.png',
 './test/007.png',
 './test/008.png',
 './test/009.png',
 './test/010.png',
 './test/011.png',
 './test/012.png',
 './test/013.png',
 './test/014.png',
 './test/015.png',
 './test/016.png',
 './test/017.png',
 './test/018.png',
 './test/019.png',
 './test/020.png',
 './test/021.png',
 './test/022.png',
 './test/023.png',
 './test/024.png',
 './test/025.png',
 './test/026.png',
 './test/027.png',
 './test/028.png',
 './test/029.png',
 './test/030.png',
 './test/031.png',
 './test/032.png',
 './test/033.png',
 './test/034.png',
 './test/035.png',
 './test/036.png',
 './test/037.png',
 './test/038.png',
 './test/039.png',
 './test/040.png',
 './test/041.png',
 './test/042.png',
 './test/043.png',
 './test/044.png',
 './test/045.png',
 './test/046.png',
 './test/047.png',
 './test/048.png',
 './test/049.png',
 './test/050.png',
 './test/051.png',
 './test/052

In [35]:
preds = []
for image in test_ls:
    image = val_preprocess(Image.open(image)).unsqueeze(0).to(device)
    text = clip.tokenize(text_label).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu()
        prob_idx = probs.argmax().cpu().item()
        preds.append(prob_idx)

In [39]:
sub_csv = pd.read_csv('./sample_submission.csv')

In [40]:
sub_csv['label'] = pd.Series(preds)

In [41]:
sub_csv

,id,label
0,TEST_000,18
1,TEST_001,17
2,TEST_002,18
3,TEST_003,4
4,TEST_004,17
...,...,...
787,TEST_787,18
788,TEST_788,17
789,TEST_789,17
790,TEST_790,17


In [42]:
map_d = dict(zip([i for i in range(0, 19)],df.label.unique().tolist()))

In [43]:
sub_csv['label'] = sub_csv['label'].map(map_d)

In [44]:
sub_csv

,id,label
0,TEST_000,damage
1,TEST_001,contamination
2,TEST_002,damage
3,TEST_003,molding
4,TEST_004,contamination
...,...,...
787,TEST_787,damage
788,TEST_788,contamination
789,TEST_789,contamination
790,TEST_790,contamination


In [47]:
k = sub_csv['label'].to_list()

In [48]:
def label_decoder(label):
    new_label =[
        '가구수정' if l=='Furniture' else
        '걸레받이수정' if l=='mopholder' else
        '곰팡이' if l=='mold' else
        '꼬임' if l=='twist' else
        '녹오염' if l=='rust pollution' else
        '들뜸' if l=='crack' else
        '면불량' if l=='cotton defect' else
        '몰딩수정' if l=='molding' else
        '반점' if l=='half spot' else
        '석고수정' if l=='plaster' else
        '오염' if l=='contamination' else
        '오타공' if l=='black spot' else
        '울음' if l=='wrinkle' else
        '이음부불량' if l=='wailing' else
        '창틀,문틀수정' if l=='bad joint' else
        '터짐' if l=='window frame' else
        '틈새과다' if l=='gap' else
        '피스' if l=='piece' else
        '훼손' if l=='damage' else
        'NaN' for l in label
    ]
    return np.array(new_label)

In [49]:
n = label_decoder(k)

In [50]:
sub_csv['label'] = pd.Series(n)

In [51]:
sub_csv['label'][0][0]

'훼'

In [52]:
sub_csv.to_csv('05_22.csv', index = False)